In [3]:
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
from transformers import BertTokenizer, BertModel
from einops import rearrange
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers import AutoTokenizer
import torch
import pandas as pd

In [4]:
gpt2 = GPT2Model.from_pretrained('gpt2', output_attentions=True, output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained('gpt2')

### Short

In [25]:
data_name = 'NP'
exogenous_variables = ['Grid Load','Wind Power']
endogenous_variable = 'Nord Pool Electricity Price'
data_collection = 'Nord Pool electricity market'

data_name = 'PJM'
exogenous_variables = ['System Load','SyZonal COMED load'] 
endogenous_variable = 'Pennsylvania-New Jersey-Maryland Electricity Price'
data_collection = 'Pennsylvania-New Jersey-Maryland market'

data_name = 'BE'
exogenous_variables = ['Generation','System Load'] 
endogenous_variable = 'Belgium\'s Electricity Price'
data_collection = 'Belgium\'s electricity market'

data_name = 'FR'
exogenous_variables = ['Generation','System Load'] 
endogenous_variable = 'France\'s Electricity Price'
data_collection = 'France\'s electricity market'

data_name = 'DE'
exogenous_variables = ['Wind power','Ampirion zonal load'] 
endogenous_variable = 'German\'s Electricity Price'
data_collection = 'German\'s electricity market'

In [26]:
exogenous_variables_text_template = 'This exogenous variable is'
endogenous_template = 'The endogenous variable is'
dataset_template1 = f'This dataset is {data_name}, containing the data collected from {data_collection}. Exogenous variables are'
dataset_template2 = f'and it is necessary to utilize these external variables sequentially to predict the endogenous variable'

In [27]:
prompts = []
for v in exogenous_variables:
    #v = variables[i]
    prompts.append(exogenous_variables_text_template+' '+ v+'.')
    dataset_template1 = dataset_template1+' '+v+','
prompts.append(endogenous_template+' '+ endogenous_variable+'.')
dataset_template2 = dataset_template2 + ' ' +  endogenous_variable+ '.'
dataset_template1 = dataset_template1 + dataset_template2
prompts.append(dataset_template1)
print(prompts)

['This exogenous variable is Wind power.', 'This exogenous variable is Ampirion zonal load.', "The endogenous variable is German's Electricity Price.", "This dataset is DE, containing the data collected from German's electricity market. Exogenous variables are Wind power, Ampirion zonal load,and it is necessary to utilize these external variables sequentially to predict the endogenous variable German's Electricity Price."]


In [28]:
text_emds = []
for p in prompts:
    tokens = tokenizer(p, padding=False, truncation=True, return_tensors="pt")
    # 2. 使用tokenizer将文本转换为模型的输入表示
    input_ids = tokens["input_ids"]
    # 3. 将输入表示传递给您的GPT-2模型，获取文本的嵌入表示
    with torch.no_grad():
        outputs = gpt2(input_ids=input_ids)
    print(outputs.last_hidden_state.shape)
    text_emds.append(outputs.last_hidden_state[:,-1])
#text_emds.insert(0, text_emds[-1])
text_emds = torch.cat(text_emds,dim=0)
print(text_emds.shape)
torch.save(text_emds, data_name+'.pt')

torch.Size([1, 8, 768])
torch.Size([1, 12, 768])
torch.Size([1, 9, 768])
torch.Size([1, 50, 768])
torch.Size([4, 768])
